In [1]:
#import sys
#sys.path.append('./')
import numpy as np
import lib.build.libry as ry
import time


from lib.rai.rai_env import RaiEnv
from lib.prediction.color_seg import find_ball


**ry-c++-log** /home/tim/workspace/git/robotics-course/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback
** INFO:/home/tim/workspace/git/robotics-course/rai/rai/Core/util.cpp:initCmdLine:602(1) ** cmd line arguments: 'rai-pybind -python'
** INFO:/home/tim/workspace/git/robotics-course/rai/rai/Core/util.cpp:initCmdLine:606(1) ** run path: '/home/tim/workspace/git/robotics-course/project'
** INFO:/home/tim/workspace/git/robotics-course/rai/rai/Core/graph.cpp:initParameters:1365(1) ** parsed parameters:
{python}


** INFO:/home/tim/workspace/git/robotics-course/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback


In [2]:
#########################################
# initialize rai simulation here        #
#########################################
Rai = RaiEnv(tau=.01,
             realEnv="scenarios/project_env.g",
             modelEnv="scenarios/project_env.g",
             useROS=False,
             initSim=False,
             initConfig=True,
             simulatorEngine=ry.SimulatorEngine.bullet,
             verboseSim=1,
             defaultCamera=False)

#########################################
# init simulation                       #
#########################################
Rai._init_simulation()


In [8]:
Rai.run_simulation(100,False)

Simulation finished


In [16]:
Rai.C.frames()
Rai.RealWorld.frames()[0]


In [9]:
F=Rai.C.feature(ry.FS.position, ['R_gripper'], order=1)


In [10]:
F.eval(Rai.C)
Rai.C.evalFeature()

**ry-c++-log** /home/tim/workspace/git/robotics-course/rai/rai/Kin/feature.cpp:getFrames:83(-2) CHECK failed: '!order' -- can't ground a order>0 feature on configuration without slices

STACK9 /home/tim/workspace/venvs/rai_env/bin/python(PyEval_EvalCode
STACK8 /home/tim/workspace/venvs/rai_env/bin/python(_PyEval_EvalCodeWithName
STACK7 /home/tim/workspace/venvs/rai_env/bin/python(_PyEval_EvalFrameDefault
STACK6 /home/tim/workspace/venvs/rai_env/bin/python() [0x50b4a7]
STACK5 /home/tim/workspace/venvs/rai_env/bin/python(_PyObject_MakeTpCall
STACK4 /home/tim/workspace/venvs/rai_env/bin/python(PyCFunction_Call
STACK3 /home/tim/workspace/git/robotics-course/project/lib/build/libry.cpython-38-x86_64-linux-gnu.so(+0xb0a90) [0x7f85ce6ada90]
STACK2 /home/tim/workspace/git/robotics-course/project/lib/build/libry.cpython-38-x86_64-linux-gnu.so(+0x81160) [0x7f85ce67e160]
STACK1 Feature::getFrames(rai::Configuration const&, unsigned int)
STACK0 rai::LogToken::~LogToken()


** ERROR:/home/tim/workspace/git/robotics-course/rai/rai/Kin/feature.cpp:getFrames:83(-2) CHECK failed: '!order' -- can't ground a order>0 feature on configuration without slices


RuntimeError: /home/tim/workspace/git/robotics-course/rai/rai/Kin/feature.cpp:getFrames:83(-2) CHECK failed: '!order' -- can't ground a order>0 feature on configuration without slices

In [ ]:
Rai.run_simulation(11)




Simulation finished


In [18]:
komo = Rai.C.komo_path(2., 20, 4., True)

komo.getT()

komo.getPathFrames()




array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-1.6       ,  0.        ,  0.8       , ...,  0.        ,
          0.70710678,  0.        ],
        [ 1.6       ,  0.        ,  0.8       , ...,  0.        ,
          0.70710678,  0.        ],
        ...,
        [ 1.55      ,  1.        ,  1.5       , ...,  0.22871307,
          0.52600405,  0.75121163],
        [ 0.        ,  0.        ,  3.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.5       ,  0.        ,  0.05      , ...,  0.        ,
          0.        ,  1.        ]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-1.6       ,  0.        ,  0.8       , ...,  0.        ,
          0.70710678,  0.        ],
        [ 1.6       ,  0.        ,  0.8       , ...,  0.        ,
          0.70710678,  0.        ],
        ...,
        [ 1.55      ,  1.        ,  1.5       , ...,  

In [7]:
# Interpolating komo


    komo = komo_lift_and_throw(Rai, gripper)

    # komo.view_play(False, 1)

    # length for ball release in last frame
    komo_length = komo.getT()
    komo_frames = komo.getPathFrames()
    komo_tau = komo.getPathTau()
    i = 0
    tau = 0.01
    sim_time = 1.5

    # execute komo path to the ball
    # for frame, tau in zip(komo.getPathFrames(), komo.getPathTau()):
    for t in range(int(sim_time / tau)):
        time.sleep(tau)
        q = Rai.S.get_q()
        # Rai.C.setJointState(q)  # set your robot model to match the real q

        i = min(komo_length - 1, int((t) *
                tau / sum(komo_tau) * komo_length))

        frame = komo_frames[i]

        Rai.C.setFrameState(frame)
        q_frame = Rai.C.getJointState()
        # interpolate to next joint state
        q += (q_frame - q) * ((t * tau % komo_tau[i]) / komo_tau[i])

        # release ball in last frame
        if t * tau > sum(komo_tau[:-3]) and Rai.S.getGripperIsGrasping(gripper):
            Rai.S.openGripper(gripper, speed=2., width=0.01)
            if not Rai.S.getGripperIsGrasping(gripper):
                print("RELEASED")
                grasped = False
                break

        # send position to the simulation
        Rai.S.step(q, tau, ry.ControlMode.position)

<bound method PyCapsule.getGripperWidth of <libry.Simulation object at 0x7f816804d970>>

In [40]:
type(Rai) is RaiEnv


True

In [4]:
Rai.S.getImageAndDepth()

(array([[[129, 140, 151],
         [129, 140, 151],
         [129, 140, 151],
         ...,
         [129, 140, 151],
         [129, 140, 151],
         [129, 140, 151]],
 
        [[129, 140, 151],
         [129, 140, 151],
         [129, 140, 151],
         ...,
         [129, 140, 151],
         [129, 140, 151],
         [129, 140, 151]],
 
        [[129, 140, 151],
         [129, 140, 151],
         [129, 140, 151],
         ...,
         [129, 140, 151],
         [129, 140, 151],
         [129, 140, 151]],
 
        ...,
 
        [[129, 140, 151],
         [129, 140, 151],
         [129, 140, 151],
         ...,
         [129, 140, 151],
         [129, 140, 151],
         [129, 140, 151]],
 
        [[129, 140, 151],
         [129, 140, 151],
         [129, 140, 151],
         ...,
         [129, 140, 151],
         [129, 140, 151],
         [129, 140, 151]],
 
        [[129, 140, 151],
         [129, 140, 151],
         [129, 140, 151],
         ...,
         [129, 140, 151],
  

In [22]:
tau = 0.01
# simulate to grasp
for t in range(int(0.3 / tau)):
    time.sleep(tau)
    q = Rai.S.get_q()
    Rai.C.setJointState(q)  # set your robot model to match the real q

    Rai.S.closeGripper("R_gripper", speed=200.)

    # send velocity controls to the simulation
    Rai.S.step(np.zeros_like(q), tau, ry.ControlMode.none)


In [ ]:
ry.Frame.

In [21]:
class PojectileMotion:
    def __init__(self, position0, t0=0, mass=None, gravity=9.81):
        self.t0 = t0  # inital time in s

        self.t = [self.t0]  # inital time in ms
        self.positions = [position0]  # positions in m
        self.velocities = [] # velocities in m/s     
        self.accelerations = []  # x accelerations in m/(s*s)

        self.m = mass  # mass
        self.g = gravity  # gravity

    def updatePosition(self, x, y, z, t):
        self.t.append(t)
        self.positions.append([x,y,z])
        self._updateVel()


    def _updateVel(self):
        for i in range(len(self.t) - 1):
            vel = []

            # skip already calulated ones
            if i < len(self.velocities):
                continue
# in all 3 dimensions
            for direction in range(3): 
                pos0 = self.positions[i][direction]
                pos1 = self.positions[i+1][direction]
                timeDelta = (self.t[i + 1] - self.t[i])
                
                vel.append((pos1 - pos0) / timeDelta)
            print(vel)
            self.velocities.append(vel)
            
        
        return self.velocities

    def getPosition(self):
        sdfgs


proj = PojectileMotion(0,0,0)

print(proj.t)

proj.updatePosition(1,0,2,1)
proj.updatePosition(1.5,1,0,2)

print(proj.t)
print(proj.velocities)




[0]


TypeError: unsupported operand type(s) for -: 'int' and 'NoneType'